In [ ]:
#그로브쉴드에 꽂은 피코를 이용하여 여러 개의 온도계를 감지하는 프로그램
#온도를 소수점 2째 자리까지만 표시하고 저장한다.
#연결 위치는 다음과 같다
#온도센서: D18 그로브 소켓
#데이터 저장 파일명: multi-temp-data.csv

import utime
import _thread

import time
import math
import machine
from machine import Pin, I2C, UART
from machine import WDT

import onewire, ds18x20


csv_file_name = "three-temp-data.csv"

# Utility function to convert a second count to hours:minutes:seconds
def secondsToString(s):
    hour = math.floor(s/3600)
    minute = math.floor((s-(hour*3600))/60)
    second = s - (60 * minute) - (3600 * hour);
    return '{:02d}:{:02d}:{:02d}'.format(hour, minute, second)

# We create a semaphore (A.K.A lock) to message between threads
baton = _thread.allocate_lock()

v_ref = 3.3
v_ref_digit = 65535
conversion_factor = v_ref / v_ref_digit
templist=[]
avgilist=[0]
prev_temp = 0

n=0

#온도 센서 연결(D16) 
ds_pin16 = machine.Pin(16) #D16에 연결(노란색: 데이타 선) 
ds_sensor16 = ds18x20.DS18X20(onewire.OneWire(ds_pin16))
roms16 = ds_sensor16.scan()
print('Found DS16 devices: ', roms16)

#온도 센서 연결(D18) 
ds_pin18 = machine.Pin(18) #D18에 연결(노란색: 데이타 선) 
ds_sensor18 = ds18x20.DS18X20(onewire.OneWire(ds_pin18))
roms18 = ds_sensor18.scan()
print('Found DS18 devices: ', roms18)

#온도 센서 연결(D20) 
ds_pin20 = machine.Pin(20) #D20에 연결(노란색: 데이타 선) 
ds_sensor20 = ds18x20.DS18X20(onewire.OneWire(ds_pin20))
roms20 = ds_sensor20.scan()
print('Found DS devices: ', roms20)


csvfile = open(csv_file_name, "w")
csvfile.write("time, temp16, temp18, temp20")
csvfile.write("\n")
print("측정 시작!")

# Store the start time to use for deriving the uptime
start = time.time()
# Initialise the Watchdog timer timout (5 seconds)
wdt = WDT(timeout=5000)

while True:
    # Reset the watchdog timer
    wdt.feed()
    runtime = (time.time() - start)
    print("uptime: " + secondsToString(runtime))
    print("Saving.. to " + csv_file_name)
       
    csvfile.write(secondsToString(runtime))
    #16
    ds_sensor16.convert_temp()
    time.sleep_ms(750)
    
    for rom in roms16:
        temp = ds_sensor16.read_temp(rom)
        str_temp = "{:.2f}".format(temp)
        avg_giulgi=(temp-prev_temp)/0.75
        print(str_temp, end=' ')
        print(avg_giulgi, end = ' ')
        templist.append(temp)
        avgilist.append(avg_giulgi)
        csvfile.write(','+ str_temp)
        prev_temp=temp
        n+=1
        if templist[-1]> -2 and -0.01<avg_giulgi and avg_giulgi<0:
            newtemp=temp
            break
    if templist[-1]> -2 and -0.01<avg_giulgi and avg_giulgi<0:
        break
        
    #18
    ds_sensor18.convert_temp()
    time.sleep_ms(750)
    for rom in roms18: 
        temp = ds_sensor18.read_temp(rom)
        str_temp = "{:.2f}".format(temp)
        print(str_temp, end=' ')
        csvfile.write(','+ str_temp)
    #20
    ds_sensor20.convert_temp()
    time.sleep_ms(750)
    for rom in roms20:
        temp = ds_sensor20.read_temp(rom)
        str_temp = "{:.2f}".format(temp)
        print(str_temp, end=' ')
        csvfile.write(','+ str_temp)
        
    print("") #줄바꾸기
    csvfile.write("\n")

for i in range(n):
    newtemp+=avgilist[-1]*0.75
    if abs(newtemp-templist[-i-1])<=0.1 and i>30:
        print('어는점은',newtemp, '도 입니다')
        break 
    
    


In [2]:
for i in range(10):
    print(i)
    if i == 5:
        break
for j in range(10):
    
print(100)

0
1
2
3
4
5
100


In [1]:
while True:
    for i in range(10):
        print(i)
        if i == 5:
            break
    if i == 5:
        break
    for j in range(11, 20):
        print(j)

0
1
2
3
4
5
